In [11]:
using JuMP, CPLEX
using Plots
using JLD

In [12]:
function new_T(node_values,lb,ub)

function my_lcm(a, b)
    return (a * b) ÷ gcd(a, b)
end

function dijkstra_lcm(graph, src, target)
    n = length(keys(graph))+1
    lcm_values = fill(Inf, n+1) 
    lcm_values[src] = 1
    pq = [(1, src)]  # Priority queue of (lcm, vertex) tuples
    visited = fill(false, n+1)
    predecessors = fill(-1, n+1)  
    
    while !isempty(pq)
        
        pq = sort(pq,rev=true)
        (current_lcm, u) = pop!(pq) 
        
        if visited[u]
            continue  
        end

        if u == target
            path = [u]
            while predecessors[u] != -1
                push!(path, predecessors[u])
                u = predecessors[u]
            end
            return current_lcm, reverse(path) 
        end

        visited[u] = true 

        for (v, weight) in graph[u]
        
            new_lcm = my_lcm(current_lcm, weight)
            if new_lcm < lcm_values[v]
                lcm_values[v] = new_lcm
                push!(pq, (new_lcm, v))
                predecessors[v] = u
            end
        end
    end

    return -1  # Target not reachable
end


function create_graph(node_values)
    n = length(node_values)
    graph = Dict()

    # Start vertex
    graph[1] = sort(Dict(1 + i => v for (i, v) in enumerate(vertices_in_range(node_values[1]))))

    # Core nodes
    for i in 1:(n-1) 
        vertices = vertices_in_range(node_values[i])
        next_vertices = vertices_in_range(node_values[i + 1])

        k0=maximum(keys(graph))
        k1=minimum(keys(graph[k0]))
        k2=maximum(keys(graph[k0]))
    
        for (i, v) in enumerate(vertices)
            graph[k1+i-1] = sort(Dict(k2 + j => u for (j, u) in enumerate(next_vertices)))
        end
    end

    # End vertex
        k0=maximum(keys(graph))
        k1=minimum(keys(graph[k0]))
        k2=maximum(keys(graph[k0]))
        
    vertices = vertices_in_range(node_values[n])  
    for (i, v) in enumerate(vertices)
        graph[k1+i-1] = Dict(k2 + 1 => 1)  
    end

    return graph
end

function vertices_in_range(pi)
    lower = ceil(pi - lb*pi)
    upper = floor(pi + ub*pi)
    vertices = [] 
    step_size = 1  

    value = lower
    while value <= upper
        push!(vertices,Int(value)) 
        value += step_size
    end
    vertices=sort(vertices)
    return vertices
end

    
graph = create_graph(node_values)
k0=maximum(keys(graph))
k1=maximum(keys(graph[k0]))
start = 1
target = k1
graph1=sort(graph)
shortest_path_lcm = dijkstra_lcm(graph, start, target)

T_new=[]
for (i,j) in enumerate(shortest_path_lcm[2][1:end-2])
    push!(T_new,graph[j][shortest_path_lcm[2][i+1]])
end

    return T_new
    
end


new_T (generic function with 1 method)

In [13]:
using LinearAlgebra
using JuMP, CPLEX
using Plots
using LinearAlgebra

file = jldopen("data2.jld", "r", mmaparrays=true) # change this value from data1 to data10
D = read(file, "D") 

n = size(D, 2)
g = gcd([D[1, :]..., D[2, :]...])
#D = D / g
C = D[1, :]
F = D[2, :]
T = sum(D, dims=1)
t = lcm(map(Int, T))
nums = Int(sum(T))
T_max = maximum(T)
println(t)


4620


In [14]:
A = zeros(n, nums)

p_matrix = []
A_matrix = []

k = 1

for i in 1:n
    A[i, k:k+Int(T[i])-1] .= 1    # equality
    k += Int(T[i])
    
    p_i = zeros(Int, Int(T[i]))
    p_i[1:Int(C[i])] .= 1
    push!(p_matrix, p_i)
end

A_bar = zeros(t, nums) # inequality

for i in 1:t
    k = 1
    for j in 1:n
        A_bar[i, k:k+Int(T[j])-1] .= circshift(p_matrix[j], i-1)
        k += Int(T[j])
    end
    
end


In [15]:
model=Model(CPLEX.Optimizer)

@variable(model,x[1:nums],Bin)
@variable(model,m)

@constraint(model,A*x.==1)
@constraint(model,A_bar*x.<=m)

@objective(model,Min,m)

optimize!(model)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
Tried aggregator 1 time.
Reduced MIP has 4630 rows, 735 columns, and 1848734 nonzeros.
Reduced MIP has 734 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.37 sec. (401.95 ticks)
Found incumbent of value 10.000000 after 0.57 sec. (615.44 ticks)
Probing time = 0.04 sec. (46.43 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 4630 rows, 735 columns, and 1848734 nonzeros.
Reduced MIP has 734 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 1.02 sec. (1607.92 ticks)
Probing time = 0.05 sec. (46.43 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 5.36 sec. (3730.57 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0          

In [16]:
value(m) # save value of m and time from previous step as without_waiting time

7.0

In [17]:
# calculation of waiting time
T_new=new_T(T,0.1,0.0)
t1 = lcm(map(Int, T_new))

N=t1./T_new  


temp=T-T_new'
for i in 1:n
    D[2,i]=D[2,i]-temp[i]
end


C = D[1, :]
F = D[2, :]
g = gcd([D[1, :]..., D[2, :]...])
#D = D / g
T = sum(D, dims=1)
t = lcm(map(Int, T))
nums = Int(sum(T))
T_max = maximum(T)

t

504

In [18]:
A = zeros(n, nums)

p_matrix = []
A_matrix = []

k = 1

for i in 1:n
    A[i, k:k+Int(T[i])-1] .= 1    # equality
    k += Int(T[i])
    
    p_i = zeros(Int, Int(T[i]))
    p_i[1:Int(C[i])] .= 1
    push!(p_matrix, p_i)
end

A_bar = zeros(t, nums) # inequality

for i in 1:t
    k = 1
    for j in 1:n
        A_bar[i, k:k+Int(T[j])-1] .= circshift(p_matrix[j], i-1)
        k += Int(T[j])
    end
    
end


In [19]:
model=Model(CPLEX.Optimizer)

@variable(model,x[1:nums],Bin)
@variable(model,m)


@constraint(model,A*x.==1)
@constraint(model,A_bar*x.<=m)


@objective(model,Min,m)

optimize!(model)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de
Tried aggregator 1 time.
Reduced MIP has 514 rows, 708 columns, and 202307 nonzeros.
Reduced MIP has 707 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.04 sec. (52.69 ticks)
Found incumbent of value 10.000000 after 0.06 sec. (76.17 ticks)
Probing time = 0.01 sec. (12.36 ticks)
Cover probing fixed 0 vars, tightened 1 bounds.
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 514 rows, 708 columns, and 202307 nonzeros.
Reduced MIP has 707 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.08 sec. (136.42 ticks)
Probing time = 0.01 sec. (9.81 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.39 sec. (931.80 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound   

In [20]:
value(m) # save value of m and time from previous step

7.0